## 1. importing libraries

In [1]:
import pandas as pd
import numpy as np
import os

## 2. importing datasets

In [61]:
# seting up folder path

path = '/Users/martamajer/Desktop/Desktop - Marta’s MacBook Air - 1/09-2024 Instacart Basket Analysis/'

# importing datasets

ords_prods_flags = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_flags.pkl'))
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [62]:
# checking information about the customer dataset

customers.head()

# customers.tail()
# customers.shape
# customers.info()
# customers.dtypes 
# customers.columns
# customers.describe()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 3. data wrangling 

In [63]:
# dropping date_join column from customers

customers = customers.drop(columns = ['date_joined'])

In [64]:
# checking NaN values within customers df
# names have a lot of NA values, around 11k

customers['First Name'].value_counts(dropna = False)

First Name
NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: count, Length: 208, dtype: int64

In [65]:
# renaming the columns

customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)
customers.rename(columns = {'Surnam' : 'surname'}, inplace = True)
customers.rename(columns = {'Gender' : 'gender'}, inplace = True)
customers.rename(columns = {'STATE' : 'state'}, inplace = True)
customers.rename(columns = {'Age' : 'age'}, inplace = True)

## 4. data consistency check 

In [67]:
# changing columns with object as a datatype to string datatype to make sure we've got text in there

customers[['first_name', 'surname', 'gender', 'state', 'fam_status']] = customers[['first_name', 'surname', 'gender', 'state', 'fam_status']].astype('string')

In [68]:
customers.dtypes 

user_id                  int64
first_name      string[python]
surname         string[python]
gender          string[python]
state           string[python]
age                      int64
n_dependants             int64
fam_status      string[python]
income                   int64
dtype: object

In [69]:
customers.shape

(206209, 9)

In [70]:
# python syntax to check mixed type columns

for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


In [ ]:
# checking the list of missing values in first_name 

missing_rows = customers[customers['first_name'].isna()]
print(missing_rows)

In [71]:
# python syntax to check mixed type columns

for col in ords_prods_flags.columns.tolist():
  weird = (ords_prods_flags[[col]].applymap(type) != ords_prods_flags[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (ords_prods_flags[weird]) > 0:
    print (col)

product_name


In [ ]:
# checking the list of missing values in product_name 

missing_rows = ords_prods_flags[ords_prods_flags['product_name'].isna()]
print(missing_rows)

### decision

for customers: first_name column has a lot of NA values, around 11k, keeping it as they are 

for ords_prods_flags and product_name: I am deciding to keep the missing values as they are - just to put the light on we need to be more percise with the data collection moving forward, otherwise we are loosing lots of insight, given that it's around 20k of those missing values in the product_name.

In [49]:
# looking for full duplicates within customers

customers_dups = customers[customers.duplicated()]
customers_dups

#### observation:

there is no duplicates in customers

## 5. merging datasets

In [ ]:
# checking if user_id has the same data type in both df

ords_prods_flags.dtypes

In [ ]:
# checking if user_id has the same data type in both df

customers.dtypes

In [78]:
# merging datasets - left table is ords_prods_flags and I am enriching this table with customers info 
ords_prods_customers = ords_prods_flags.merge(customers, on = 'user_id', how = 'inner', indicator = True)

In [79]:
# checking if merging was successful 

ords_prods_customers['_merge'].value_counts()

_merge
both          32434212
left_only            0
right_only           0
Name: count, dtype: int64

In [ ]:
ords_prods_customers.head()

## 6. exporting merged df to pickle format

In [82]:
# exporting data to pkl

ords_prods_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_customers.pkl'))